****Trabalho Prático 1 - Exercício 2****

**Grupo 1**:

* Bárbara Freixo - PG49169
* Gonçalo Soares - PG50393

**Enunciado do Problema**


Use o “package” Cryptography para

    i. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.

    ii. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

**Descrição do Problema**

Este problema consiste em implementar um sistema de criptografia assíncrono que garanta confidencialidade e autenticação. Para isso, é necessário usar o pacote Cryptography para implementar uma AEAD (Autenticado Encryption with Associated Data) com "Tweakable Block Ciphers". A cifra por blocos primitiva a ser usada é o AES-256.

Além disso, deve ser construído um canal privado de informação assíncrona, utilizando o X448 key exchange para o acordo de chaves e o Ed448 Signing & Verification para autenticação dos agentes. O sistema deve incluir uma fase de confirmação da chave acordada, garantindo que as chaves acordadas são autênticas e foram geradas pelos agentes legítimos.

**Abordagem e código da solução do ponto i.**

Antes da implementação do código foi instalado o package cryptography essencial à resolução do problema.

In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


Depois de instalado o package começamos por fazer os imports necessários. Utilizámos o módulo cryptography do Python. Os imports utilizados foram: 

* from cryptography.hazmat.primitives.ciphers.aead import AESGCM: Importa os algoritmos de cifragem AEAD AES-GCM, que é uma cifra simétrica segura que fornece confidencialidade e autenticação de dados.

* from cryptography.hazmat.primitives import hashes: Importa funções hash criptográficas, como SHA256, SHA384, SHA512, que são usadas para gerar valores hash seguros de mensagens e chaves.

* from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC: Importa a função de derivação de chave PBKDF2HMAC, que é um algoritmo de derivação de chave baseado em password usado para gerar chaves criptográficas a partir de passwords de usuário.

* from cryptography.hazmat.backends import default_backend: Importa o backend padrão da biblioteca "cryptography.hazmat", que é usado para fornecer uma implementação adequada para as funções de criptografia e hash.

* import os: Importa o módulo os, que é usado para gerar valores aleatórios de passwords, sal, chaves, nonces, etc. Além disso, também fornece funções para trabalhar com diretórios, arquivos e variáveis de ambiente do sistema operacional.

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import os

Em seguida, são definidas algumas constantes e variáveis. block_cipher é o algoritmo de cifragem de bloco usado, block_size é o tamanho do bloco do algoritmo, password e salt são usados para gerar uma chave derivada por meio de um Key Derivation Function (KDF) PBKDF2, key_len é o comprimento da chave derivada e iterations é o número de iterações do PBKDF2. O KDF é usado para transformar uma password numa chave adequada para criptografia simétrica.

In [3]:
block_cipher = algorithms.AES
block_size = block_cipher.block_size // 8  # 16 bytes

password = os.urandom(16) # random password
salt = os.urandom(16) # random salt
iterations = 100000
key_len = block_size * 2  # 32 bytes

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=key_len,
    salt=salt,
    iterations=iterations,
    backend=default_backend(),
)

key = kdf.derive(password)

Em seguida, são definidas as funções ***generate_tweak*** e ***aead_encrypt*** e ***aead_decrypt*** que são usadas para cifrar e decifrar a mensagem.

***generate_tweak*** é usada para gerar a chave de tweak. O tweak é um valor aleatório usado para modificar o comportamento da criptografia. Ele é cifrado usando o modo de criptografia Eletronic Codebook (ECB) com a chave derivada da senha. A função retorna o primeiro bloco da saída da criptografia, que é usado como a chave de tweak.

***aead_encrypt*** é usada para cifrar uma mensagem usando a cifragem GCM do AES. A mensagem é cifrada com a chave derivada da password e a chave de tweak. A autenticação de dados associados (AD) é adicionada usando o método authenticate_additional_data, e a criptografia é finalizada com o método finalize. A função retorna o texto cifrado e a tag de autenticação.

***aead_decrypt*** é usada para decifrar uma mensagem cifrada com aead_encrypt. A mensagem é decifrada usando a chave derivada da password e a chave de tweak. A autenticação de dados associados (AD) é adicionada e a decifragem é finalizada. A função retorna o texto decifrado.

In [4]:
def generate_tweak(key, tweak):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    tweak_pad = b"\x00" * (block_size - len(tweak))
    tweak += tweak_pad
    tweak_enc = encryptor.update(tweak) + encryptor.finalize()
    tweak_key = tweak_enc[:block_size]
    return tweak_key

def aead_encrypt(key, tweak, nonce, plaintext, associated_data=b""):
    cipher = Cipher(block_cipher(generate_tweak(key, tweak)), modes.GCM(nonce), backend=default_backend())
    encryptor = cipher.encryptor()
    encryptor.authenticate_additional_data(associated_data)
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    tag = encryptor.tag
    return ciphertext, tag

def aead_decrypt(key, tweak, nonce, ciphertext, tag, associated_data=b""):
    cipher = Cipher(block_cipher(generate_tweak(key, tweak)), modes.GCM(nonce, tag), backend=default_backend())
    decryptor = cipher.decryptor()
    decryptor.authenticate_additional_data(associated_data)
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

Para exemplificar o funcionamento da solução, iremos mostrar na próxima secção um exemplo de aplicação. 

**Exemplos e testes de aplicação**

Exemplo: Cifragem e Decifragem de uma mensagem curta

O código gera uma chave mestra aleatória e um nonce aleatório. Ele cifra e decifra a mensagem "estruturas criptográficas!" usando ***aead_encrypt*** e ***aead_decrypt***, respectivamente. O texto cifrado, a tag de autenticação e o texto decifrado são impressos na tela para verificar se a cifragem e decifragem funcionam corretamente.

In [5]:
key = os.urandom(32) # random master key
tweak = os.urandom(block_size) # random tweak
nonce = os.urandom(12) # random nonce
plaintext = b"estruturas criptograficas!"
ciphertext, tag = aead_encrypt(key, tweak, nonce, plaintext)
decrypted_plaintext = aead_decrypt(key, tweak, nonce, ciphertext, tag)

print("Plaintext:", plaintext)
print("Ciphertext:", ciphertext)
print("Tag:", tag)
print("Decrypted plaintext:", decrypted_plaintext)

Plaintext: b'estruturas criptograficas!'
Ciphertext: b't\xf0a\x12\xde>\x18\'Du8a\xbe"z_\x01\x16\x1d\xc0,\xfe\x17O7H'
Tag: b'x\xcb\xe90\xee\xa3\x94\x0eNwDw\xdd8\xddZ'
Decrypted plaintext: b'estruturas criptograficas!'


**Abordagem e código da solução do ponto ii.**

Este código implementa um protocolo de criptografia de chave pública em duas etapas, onde duas partes (Alice e Bob) concordam numa chave compartilhada que é usada para cifrar e decifrar mensagens. A criptografia é realizada usando as primitivas criptográficas X448 e AES-GCM, e a autenticação é realizada usando Ed25519.

Começamos por fazer os imports necessários. Os imports foram os seguintes: 

* from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey, X448PublicKey: permitem a geração de chaves privadas e públicas para o algoritmo X448.
* from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey, Ed25519PublicKey: permitem a geração de chaves privadas e públicas para o algoritmo Ed25519. 
* from cryptography.hazmat.primitives.kdf.hkdf import HKDF: é usado para derivar uma chave simétrica a partir da chave compartilhada gerada pelo acordo de chaves.
* from cryptography.hazmat.primitives.ciphers.aead import AESGCM: é usado para cifrar e decifrar a mensagem usando a chave simétrica derivada.
* from cryptography.hazmat.primitives import hashes: é usado para escolher o algoritmo de hash usado pelo HKDF.
* import os: é usado para gerar um nonce aleatório usado pelo AES-GCM.



In [10]:
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey, X448PublicKey
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey, Ed25519PublicKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
import os

A seguir, o código gera as chaves X448 para a Alice e  para o Bob e as chaves Ed25519 para autenticação:

In [15]:
# Gerando as chaves X448 para o acordo de chaves
alice_x448_private_key = X448PrivateKey.generate()
alice_x448_public_key = alice_x448_private_key.public_key()

bob_x448_private_key = X448PrivateKey.generate()
bob_x448_public_key = bob_x448_private_key.public_key()

# Gerando as chaves Ed25519 para autenticação
alice_ed25519_private_key = Ed25519PrivateKey.generate()
alice_ed25519_public_key = alice_ed25519_private_key.public_key()

bob_ed25519_private_key = Ed25519PrivateKey.generate()
bob_ed25519_public_key = bob_ed25519_private_key.public_key()

Em seguida, as duas partes realizam o acordo de chaves usando o protocolo de Diffie-Hellman X448:

In [16]:
# Realizando o acordo de chaves usando X448
shared_key = alice_x448_private_key.exchange(bob_x448_public_key)

O resultado é uma chave secreta compartilhada entre Alice e Bob. Em seguida, o código deriva a chave usando o algoritmo HKDF:

In [17]:
# Derivando a chave com HKDF
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)

O resultado é uma chave derivada de 32 bytes. O código então verifica se as chaves acordadas são iguais usando uma operação de criptografia AES-GCM:

In [18]:
# Verificando se as chaves acordadas são iguais
nonce = os.urandom(12)
challenge = os.urandom(16)

aesgcm = AESGCM(derived_key[:32])
encrypted_challenge = aesgcm.encrypt(nonce, challenge, None)
decrypted_challenge = aesgcm.decrypt(nonce, encrypted_challenge, None)

if challenge == decrypted_challenge:
    print('Chave acordada confirmada com sucesso')
else:
    print('Erro na confirmação da chave acordada')

Chave acordada confirmada com sucesso


Se a chave acordada for igual, o código exibe uma mensagem confirmando o sucesso da operação. Em seguida, o código usa a chave derivada para cifrar e decifrar uma mensagem usando AES-GCM:

In [20]:
# Cifrando a mensagem com AES-GCM
nonce = os.urandom(12)
aad = b'authenticated but unencrypted data'
message = b'Mensagem secreta'

aesgcm = AESGCM(derived_key[:32])
ciphertext = aesgcm.encrypt(nonce, message, aad)

# Decifrando a mensagem com AES-GCM
aesgcm = AESGCM(derived_key[:32])
plaintext = aesgcm.decrypt(nonce, ciphertext, aad)

Por fim, o código assina a mensagem usando a chave privada Ed25519 de Alice e verifica a assinatura usando a chave pública correspondente e exibe o resultado da crifragem e decifragem da mensagem:

In [22]:
# Assinando a mensagem com Ed25519
signature = alice_ed25519_private_key.sign(message)

# Verificando a assinatura com a chave pública correspondente
try:
    alice_ed25519_public_key.verify(signature, message)
    print('Assinatura válida')
except:
    print('Assinatura inválida')
    
# Exibindo as chaves geradas
print('Chaves geradas:')
print(f'Chave privada X448 de Alice: {alice_x448_private_key.private_bytes(Encoding.Raw, PrivateFormat.Raw, NoEncryption())}')
print(f'Chave pública X448 de Alice: {alice_x448_public_key.public_bytes(Encoding.Raw, PublicFormat.Raw)}')
print(f'Chave privada Ed25519 de Alice: {alice_ed25519_private_key.private_bytes(Encoding.Raw, PrivateFormat.Raw, NoEncryption())}')
print(f'Chave pública Ed25519 de Alice: {alice_ed25519_public_key.public_bytes(Encoding.Raw, PublicFormat.Raw)}')
print(f'Chave privada X448 de Bob: {bob_x448_private_key.private_bytes(Encoding.Raw, PrivateFormat.Raw, NoEncryption())}')
print(f'Chave pública X448 de Bob: {bob_x448_public_key.public_bytes(Encoding.Raw, PublicFormat.Raw)}')
print(f'Chave privada Ed25519 de Bob: {bob_ed25519_private_key.private_bytes(Encoding.Raw, PrivateFormat.Raw, NoEncryption())}')
print(f'Chave pública Ed25519 de Bob: {bob_ed25519_public_key.public_bytes(Encoding.Raw, PublicFormat.Raw)}')

Assinatura válida
Chaves geradas:
Chave privada X448 de Alice: b'P\x98\x98\xac5b\xbb\xff\xf1\xfd\xb8MtD{\xf9\x18;\x7f\xf4\xa5\xe3dB\x86as\xb0tC?^\x04\xcfi\x8e~\x14\x8f\x0f\xb8"a\xc4\xa4\x86Hn\xdb:oA]\x89S\xeb'
Chave pública X448 de Alice: b'\xe3\xe2\xaa\xe1-\xfcS=\xa5\xe5M\xbc\x97\xa8$s\xad\xe4xrv\x8d\x80\xeb3\xdf\x1e\xfb\xcc0\xfb#5\xac\x16\x82\xf7\x05)\xb0\x16\x12\xa9E\x9e\xfc\x0c*\x950\xd3\xf8\x9f\xddA5'
Chave privada Ed25519 de Alice: b'\xca\xf2Kr\xee\xb8\xd8~\xd4\xa7H>O\x92\xd4\xc3\xe2\xb3\r5\xe9Xc7\xbd\xa7\xfaI\x86]\x8f\xfc'
Chave pública Ed25519 de Alice: b'\xf7\x17V\x01\xfb\xe5L\xbe\xdf(\x9b\xa3\x84Om##\xf3\xaf\xf4\xa8\xc2\x84%\x13\x9e\x9b\x92\xebo\xeaA'
Chave privada X448 de Bob: b'\xbc\xff\xa5o\x89\xc7$I\x92\xe5\x93\x0b\x1e3cr\xact\xbfe\x82u\x11\xec3\xe9\x9an\xf5oq!\xa8I\xab\xdc\xf2W\x9bq\x8a\xe2\xb8\x8bIOA\xddM\xf0h\x86u\x14\x8d\x9e'
Chave pública X448 de Bob: b'w\xab\x96\xf5lu\xb9\xfa\xa8\x95\x9fYp\xdb\xf5\\*\x82\xe8"\xe6\x9d\xfd\x8b\xc0\x80\xe2(\xe6\x8f\x85rR[6x\r\x97\xe0\x